In [1]:
#install tff
!pip install tensorflow_federated

     |████████████████████████████████| 602 kB 9.2 MB/s 
     |████████████████████████████████| 192 kB 40.1 MB/s 
     |████████████████████████████████| 172 kB 47.0 MB/s 
     |████████████████████████████████| 454.4 MB 21 kB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 4.0 MB 42.5 MB/s 
     |████████████████████████████████| 462 kB 41.9 MB/s 
     |████████████████████████████████| 1.2 MB 20.1 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68726 sha256=424b440581e054c1d243c61533010ce03d93b267f763c4e42d7dcb97ced940d8
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: cachetools
    Fou

In [2]:
#install flower nightly release
!pip install git+https://github.com/adap/flower.git@release/0.17#egg=flwr["simulation"]  # For a specific branch (release/0.17) w/ extra ("simulation")

  Cloning https://github.com/adap/flower.git (to revision release/0.17) to /tmp/pip-install-v84i63yj/flwr_bcf5940be931494f9f140ac1783d50f0
  Running command git clone -q https://github.com/adap/flower.git /tmp/pip-install-v84i63yj/flwr_bcf5940be931494f9f140ac1783d50f0
  Running command git checkout -b release/0.17 --track origin/release/0.17
  Switched to a new branch 'release/0.17'
  Branch 'release/0.17' set up to track remote branch 'release/0.17' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 49.6 MB 1.3 MB/s 
     |████████████████████████████████| 119 kB 64.2 MB/s 
     |████████████████████████████████| 1.1 MB 59.8 MB/s 
     |████████████████████████████████| 128 kB 66.3 MB/s 
     |████████████████████████████████| 65 kB 3.8 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 201 kB 46.5 MB/s 
  

In [3]:
#install tf_privacy
!pip install tensorflow_privacy

In [3]:
#mount drive if running from colab
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [14]:
#imports and load data from sql server
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow_federated as tff
from datetime import datetime
import collections
import tensorflow as tf
import math
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras_vectorized import (
    VectorizedDPKerasAdamOptimizer, VectorizedDPKerasSGDOptimizer
)
import numpy as np
from typing import Any, Callable, Dict, List, Optional, Tuple
import pandas as pd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import tensorflow_privacy
from time import time
from tensorflow.keras.callbacks import History 


import tensorflow_privacy
from sklearn.model_selection import train_test_split


import flwr as fl
tff.simulation.datasets.emnist.load_data(only_digits=True)

(<tensorflow_federated.python.simulation.datasets.serializable_client_data.PreprocessSerializableClientData at 0x7fa6efbaaa90>,
 <tensorflow_federated.python.simulation.datasets.serializable_client_data.PreprocessSerializableClientData at 0x7fa6f0053d50>)

In [27]:
#class defining individual flower client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val, cid) -> None:

        #Add Differential Privacy to Model
        temp = model
        l2_norm_clip = 1
        noise_multiplier = NOISE
        num_microbatches = MICROBATCH_NUM

        optimizer =  VectorizedDPKerasSGDOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=num_microbatches, learning_rate = LEARNING_RATE)

        loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)

        temp.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

        X_val, X_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=42)

        self.model = temp
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = X_val, y_val
        self.x_test, self.y_test = X_test, y_test
        self.cid = cid

    def get_parameters(self):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        prev_loss, _ = self.model.evaluate(self.x_test, self.y_test, verbose=2)
        hist = self.model.fit(self.x_train, self.y_train, epochs=EPOCHS, batch_size = BATCH_SIZE, verbose=2)
        cur_loss = hist.history['loss'][0]
        k_param = prev_loss / cur_loss

        return self.model.get_weights(), len(self.x_train), {"metrics": k_param}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)

        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)

        f = open("client_logs.txt", "a")
        time = datetime.now().time()
        f.write(str(time) + " " + self.cid + " " + str(len(self.x_train)) + " " + str(loss) + " " + str(acc))
        f.write('\n')
        f.close()

        return loss, len(self.x_val), {"accuracy": acc}

In [5]:
#data processing
@tf.autograph.experimental.do_not_convert
def preprocess(dataset, batch_size):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 28,28]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.shuffle(SHUFFLE_BUFFER, seed=1).batch(
      batch_size).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [6]:
#load in specific partition of dataset based on client id
def load_partition(idx: int):
    """Load 1/10th of the training and test data to simulate a partition."""

    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)
    client_train = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[idx])
    client_test = emnist_test.create_tf_dataset_for_client(emnist_test.client_ids[idx])

    processed_train = preprocess(client_train, len(list(client_train)))
    processed_test = preprocess(client_test, len(list(client_test)))

    sample_train = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(processed_train)))

    sample_test = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(processed_test)))

    x_train = sample_train['x']
    y_train = sample_train['y']

    x_test = sample_test['x']
    y_test = sample_test['y']

    #Reshape to proper dimensions
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

    #If number of overall training samples not evenly divided by batch size. Because every batch needs to be evenly divided by Microbatch_Num
    if x_train.shape[0] % BATCH_SIZE != 0:
      drop_num = x_train.shape[0] % BATCH_SIZE
      x_train = x_train[:-drop_num]
      y_train = y_train[:-drop_num]
    
    if x_train.shape[0] % BATCH_SIZE != 0:
                raise ValueError(
                    "Batch Size should divide total train samples"
                )
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

    
    y_train = np.array(y_train, dtype=np.int32)
    y_test = np.array(y_test, dtype=np.int32)

    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

    return (x_train,y_train),(x_test,y_test)

In [7]:
#gets model and data for a specific client
def client_fn(cid: str) -> fl.client.Client:
    # Create model

    model = create_model()

    #load partition
    (x_train, y_train), (x_val, y_val) = load_partition(int(cid))

    # Create and return client
    return FlowerClient(model, x_train, y_train, x_val, y_val, cid)

In [8]:
#Returns Model for Client and Server Side.
def create_model():
    model = tf.keras.Sequential()
    # 32 convolution filters used each of size 3x3
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    # 64 convolution filters used each of size 3x3
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # choose the best features via pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # randomly turn neurons on and off to improve convergence
    model.add(Dropout(0.25))  
    # flatten since too many dimensions, we only want a classification output
    model.add(Flatten())
    # fully connected to get all relevant data
    model.add(Dense(128, activation='relu'))
    # one more dropout
    model.add(Dropout(0.5))
    # output a softmax to squash the matrix into output probabilities
    model.add(Dense(10))
    return model

In [26]:
#server-side evaluation of our learned models
def get_eval_fn(model, num_clients):
    """Return an evaluation function for server-side evaluation."""

    x_vals = np.empty((0,28,28,1))
    y_vals = np.empty((0,10))

    for i in range(num_clients):
        emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)
        client_test = emnist_test.create_tf_dataset_for_client(emnist_train.client_ids[i])


        processed_client_test = preprocess(client_test, len(list(client_test)))

        sample_test = tf.nest.map_structure(lambda x: x.numpy(),
                                         next(iter(processed_client_test)))
        x = sample_test['x']
        x = x.reshape(x.shape[0], 28, 28, 1)
        
        y = np.array(sample_test['y'], dtype=np.int32)

        y = tf.keras.utils.to_categorical(y, num_classes=10)

        #evenly split overall test set into validation and final test set
        X_val, X_test, y_val, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

        x_vals = np.append(x_vals, X_test, axis=0)
        y_vals = np.append(y_vals, y_test, axis=0)


    # The `evaluate` function will be called after every round
    def evaluate(weights: fl.common.Weights,) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        model.set_weights(weights)  # Update model with the latest parameters
        loss, accuracy = model.evaluate(x_vals, y_vals)
        num_examples = len(x_vals)
        print('')
        print('server accuracy ' + str(accuracy))
        print('server loss ' + str(loss))
        print('')

        f = open("server_logs.txt", "a")
        time = datetime.now().time()
        f.write(str(time) + " " + str(num_examples) + " " + str(loss) + " " + str(accuracy))
        f.write('\n')
        f.close()

        return loss, {"accuracy": accuracy}
        

    return evaluate

In [10]:
from logging import WARNING
from typing import Callable, Dict, List, Optional, Tuple

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    Weights,
    parameters_to_weights,
    weights_to_parameters,
)
from flwr.common.logger import log
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy


#define custom FedAvg class so we can have client-side and server-side evaluation
class custom_FedAvg(fl.server.strategy.FedAvg):
  def configure_evaluate(
    self, rnd: int, parameters: Parameters, client_manager: ClientManager
) -> List[Tuple[ClientProxy, EvaluateIns]]:
    """Configure the next round of evaluation."""
    # Do not configure federated evaluation if a centralized evaluation
    # function is provided

    # Parameters and config
    config = {}
    if self.on_evaluate_config_fn is not None:
        # Custom evaluation config function provided
        config = self.on_evaluate_config_fn(rnd)
    evaluate_ins = EvaluateIns(parameters, config)

    # Sample clients
    if rnd >= 0:
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
    else:
        clients = list(client_manager.all().values())

    # Return client/config pairs
    return [(client, evaluate_ins) for client in clients]

In [31]:
#define hyper-parameters

NUM_CLIENTS = 100

#preprocessing
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

#training
BATCH_SIZE = 8
MICROBATCH_NUM = 4
EPOCHS = 5
ROUNDS = 150

#FOR SGD
LEARNING_RATE = .1
NOISE = .2

In [22]:
model = create_model()

model.compile(optimizer = 'sgd', loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Create FedAvg strategy
strategy=custom_FedAvg(
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_eval=0.1,  # Sample 10% of available clients for evaluation
        min_fit_clients=10,  # Never sample less than 10 clients for training
        min_eval_clients=10,  # Never sample less than 10 clients for evaluation
        min_available_clients=int(NUM_CLIENTS * 0.75),  # Wait until at least 75 clients are available
        eval_fn=get_eval_fn(model,NUM_CLIENTS))

# Start simulation
start = time()

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=ROUNDS,
    strategy=strategy)

end = time()

print("wall clock time was: " + str(end-start))

INFO flower 2021-12-08 20:45:13,196 | app.py:95 | Ray initialized with resources: {'object_store_memory': 3708386918.0, 'memory': 7416773838.0, 'CPU': 2.0, 'node:172.28.0.2': 1.0}
INFO:flower:Ray initialized with resources: {'object_store_memory': 3708386918.0, 'memory': 7416773838.0, 'CPU': 2.0, 'node:172.28.0.2': 1.0}
INFO flower 2021-12-08 20:45:13,208 | app.py:104 | Starting Flower simulation running: {'num_rounds': 50}
INFO:flower:Starting Flower simulation running: {'num_rounds': 50}
INFO flower 2021-12-08 20:45:13,214 | server.py:118 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2021-12-08 20:45:13,221 | server.py:304 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
INFO flower 2021-12-08 20:45:17,009 | server.py:307 | Received initial parameters from one random client
INFO:flower:Received initial parameters from one random client
INFO flower 2021-12-08 20:45:17,017 |

17/17 [==============================] - 1s 22ms/step - loss: 2.3114 - accuracy: 0.1084


INFO flower 2021-12-08 20:45:17,755 | server.py:127 | initial parameters (loss, other metrics): 2.311387538909912, {'accuracy': 0.10841121524572372}
INFO:flower:initial parameters (loss, other metrics): 2.311387538909912, {'accuracy': 0.10841121524572372}
INFO flower 2021-12-08 20:45:17,762 | server.py:133 | FL starting
INFO:flower:FL starting
DEBUG flower 2021-12-08 20:45:17,766 | server.py:255 | fit_round: strategy sampled 10 clients (out of 100)
DEBUG:flower:fit_round: strategy sampled 10 clients (out of 100)



server accuracy 0.10841121524572372
server loss 2.311387538909912

(pid=49656) 3/3 - 1s - loss: 2.3092 - accuracy: 0.0938
(pid=49656) Epoch 1/5
(pid=49656) 12/12 - 6s - loss: 2.3039 - accuracy: 0.1458
(pid=49656) Epoch 2/5
(pid=49655) 3/3 - 1s - loss: 2.3094 - accuracy: 0.0568
(pid=49655) Epoch 1/5
(pid=49656) 12/12 - 2s - loss: 2.3483 - accuracy: 0.1042
(pid=49656) Epoch 3/5
(pid=49656) 12/12 - 2s - loss: 2.3151 - accuracy: 0.1146
(pid=49656) Epoch 4/5
(pid=49656) 12/12 - 2s - loss: 2.3425 - accuracy: 0.1354
(pid=49656) Epoch 5/5
(pid=49656) 12/12 - 2s - loss: 2.3198 - accuracy: 0.1458
(pid=49655) 11/11 - 8s - loss: 2.3331 - accuracy: 0.0909
(pid=49655) Epoch 2/5
(pid=49656) 3/3 - 1s - loss: 2.3015 - accuracy: 0.1125
(pid=49656) Epoch 1/5
(pid=49655) 11/11 - 2s - loss: 2.3247 - accuracy: 0.1136
(pid=49655) Epoch 3/5
(pid=49655) 11/11 - 2s - loss: 2.3252 - accuracy: 0.0682
(pid=49655) Epoch 4/5
(pid=49655) 11/11 - 2s - loss: 2.3054 - accuracy: 0.1136
(pid=49655) Epoch 5/5
(pid=49655) 

DEBUG flower 2021-12-08 20:46:41,417 | server.py:264 | fit_round received 10 results and 0 failures
DEBUG:flower:fit_round received 10 results and 0 failures


...

(pid=49656) 13/13 - 1s - loss: 0.6015 - accuracy: 0.8654
17/17 [==============================] - 0s 27ms/step - loss: 0.3735 - accuracy: 0.8972


INFO flower 2021-12-08 21:58:43,302 | server.py:154 | fit progress: (50, 0.37353578209877014, {'accuracy': 0.8971962332725525}, 4405.536821579)
INFO:flower:fit progress: (50, 0.37353578209877014, {'accuracy': 0.8971962332725525}, 4405.536821579)
DEBUG flower 2021-12-08 21:58:43,309 | server.py:205 | evaluate_round: strategy sampled 10 clients (out of 100)
DEBUG:flower:evaluate_round: strategy sampled 10 clients (out of 100)



server accuracy 0.8971962332725525
server loss 0.37353578209877014

(pid=49656) 1/1 - 0s - loss: 0.0581 - accuracy: 1.0000
(pid=49655) 1/1 - 0s - loss: 0.0108 - accuracy: 1.0000
(pid=49656) 1/1 - 0s - loss: 0.0088 - accuracy: 1.0000
(pid=49655) 1/1 - 0s - loss: 0.0248 - accuracy: 1.0000
(pid=49655) 1/1 - 0s - loss: 0.1628 - accuracy: 1.0000
(pid=49656) 1/1 - 1s - loss: 0.4602 - accuracy: 0.8333
(pid=49655) 1/1 - 0s - loss: 0.0189 - accuracy: 1.0000
(pid=49656) 1/1 - 0s - loss: 0.0924 - accuracy: 1.0000


DEBUG flower 2021-12-08 21:58:49,057 | server.py:214 | evaluate_round received 10 results and 0 failures
DEBUG:flower:evaluate_round received 10 results and 0 failures
INFO flower 2021-12-08 21:58:49,069 | server.py:172 | FL finished in 4411.303742076001
INFO:flower:FL finished in 4411.303742076001
INFO flower 2021-12-08 21:58:49,077 | app.py:119 | app_fit: losses_distributed [(1, 2.3026251042330705), (2, 2.2982006364939163), (3, 2.2927895632657136), (4, 2.286186867290073), (5, 2.270562543318822), (6, 2.2277554704787885), (7, 2.1380289050768004), (8, 2.113801584793971), (9, 1.9443543452136922), (10, 1.7923873233795167), (11, 1.6592216383327136), (12, 1.5982926487922668), (13, 1.269535709310461), (14, 1.2138216090652179), (15, 0.9888656437397003), (16, 0.8298394876718521), (17, 0.7950358676910401), (18, 0.726173658337858), (19, 0.7740241828954445), (20, 0.6626509519255891), (21, 0.6563025387850675), (22, 0.8600374145003465), (23, 0.8622041348294884), (24, 0.4653914555344941), (25, 0.476

(pid=49655) 1/1 - 0s - loss: 0.0088 - accuracy: 1.0000
wall clock time was: 4419.672544002533


In [17]:
class custom_QFedAvg(fl.server.strategy.QFedAvg):
  def configure_evaluate(
    self, rnd: int, parameters: Parameters, client_manager: ClientManager
) -> List[Tuple[ClientProxy, EvaluateIns]]:
    """Configure the next round of evaluation."""
    # Do not configure federated evaluation if a centralized evaluation
    # function is provided

    # Parameters and config
    config = {}
    if self.on_evaluate_config_fn is not None:
        # Custom evaluation config function provided
        config = self.on_evaluate_config_fn(rnd)
    evaluate_ins = EvaluateIns(parameters, config)

    # Sample clients
    if rnd >= 0:
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
    else:
        clients = list(client_manager.all().values())

    # Return client/config pairs
    return [(client, evaluate_ins) for client in clients]

In [32]:
model = create_model()

model.compile(optimizer = 'sgd', loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Create QFedAvg strategy
strategy=custom_QFedAvg(
        q_param = 0.001,
        qffl_learning_rate = .1,
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_eval=0.1,  # Sample 10% of available clients for evaluation
        min_fit_clients=10,  # Never sample less than 10 clients for training
        min_eval_clients=10,  # Never sample less than 5 clients for evaluation
        min_available_clients=int(NUM_CLIENTS * 0.75),  # Wait until at least 75 clients are available
        eval_fn=get_eval_fn(model,NUM_CLIENTS))

# Start simulation
start = time()

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=ROUNDS,
    strategy=strategy)

end = time()

print("wall clock time was: " + str(end-start))

INFO flower 2021-12-09 06:23:36,960 | app.py:95 | Ray initialized with resources: {'object_store_memory': 3698910412.0, 'node:172.28.0.2': 1.0, 'CPU': 2.0, 'memory': 7397820827.0}
INFO:flower:Ray initialized with resources: {'object_store_memory': 3698910412.0, 'node:172.28.0.2': 1.0, 'CPU': 2.0, 'memory': 7397820827.0}
INFO flower 2021-12-09 06:23:36,971 | app.py:104 | Starting Flower simulation running: {'num_rounds': 150}
INFO:flower:Starting Flower simulation running: {'num_rounds': 150}
INFO flower 2021-12-09 06:23:36,980 | server.py:118 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2021-12-09 06:23:36,990 | server.py:304 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
INFO flower 2021-12-09 06:23:42,011 | server.py:307 | Received initial parameters from one random client
INFO:flower:Received initial parameters from one random client
INFO flower 2021-12-09 06:23:42,018

19/19 [==============================] - 1s 21ms/step - loss: 2.3055 - accuracy: 0.1085


INFO flower 2021-12-09 06:23:42,800 | server.py:127 | initial parameters (loss, other metrics): 2.305520534515381, {'accuracy': 0.10851418972015381}



server accuracy 0.10851418972015381
server loss 2.305520534515381



INFO:flower:initial parameters (loss, other metrics): 2.305520534515381, {'accuracy': 0.10851418972015381}
INFO flower 2021-12-09 06:23:42,809 | server.py:133 | FL starting
INFO:flower:FL starting
DEBUG flower 2021-12-09 06:23:42,835 | server.py:255 | fit_round: strategy sampled 10 clients (out of 100)
DEBUG:flower:fit_round: strategy sampled 10 clients (out of 100)


...

INFO flower 2021-12-09 08:52:27,432 | server.py:154 | fit progress: (103, 1.1793633699417114, {'accuracy': 0.8146911263465881}, 8924.619551005999)
INFO:flower:fit progress: (103, 1.1793633699417114, {'accuracy': 0.8146911263465881}, 8924.619551005999)
DEBUG flower 2021-12-09 08:52:27,440 | server.py:205 | evaluate_round: strategy sampled 10 clients (out of 100)
DEBUG:flower:evaluate_round: strategy sampled 10 clients (out of 100)



server accuracy 0.8146911263465881
server loss 1.1793633699417114

(pid=131342) 1/1 - 0s - loss: 1.3812 - accuracy: 0.6667
(pid=131343) 1/1 - 0s - loss: 1.4977 - accuracy: 0.5000
(pid=131342) 1/1 - 0s - loss: 1.3752 - accuracy: 0.8000
(pid=131343) 1/1 - 0s - loss: 1.5779 - accuracy: 0.5000
(pid=131342) 1/1 - 0s - loss: 1.4977 - accuracy: 0.4000
(pid=131343) 1/1 - 0s - loss: 1.0942 - accuracy: 0.7500
(pid=131342) 1/1 - 0s - loss: 1.1347 - accuracy: 1.0000
(pid=131343) 1/1 - 0s - loss: 1.2196 - accuracy: 0.8333


DEBUG flower 2021-12-09 08:52:32,311 | server.py:214 | evaluate_round received 10 results and 0 failures
DEBUG:flower:evaluate_round received 10 results and 0 failures
DEBUG flower 2021-12-09 08:52:32,336 | server.py:255 | fit_round: strategy sampled 10 clients (out of 100)
DEBUG:flower:fit_round: strategy sampled 10 clients (out of 100)


(pid=131342) 1/1 - 0s - loss: 1.1732 - accuracy: 0.7500
(pid=131343) 1/1 - 0s - loss: 1.1836 - accuracy: 1.0000
(pid=131343) 1/1 - 0s - loss: 1.1050 - accuracy: 0.8571
(pid=131343) Epoch 1/5
(pid=131342) 1/1 - 0s - loss: 1.4101 - accuracy: 0.7143
(pid=131342) Epoch 1/5
(pid=131342) 13/13 - 6s - loss: 1.4147 - accuracy: 0.5673
(pid=131342) Epoch 2/5
(pid=131343) 13/13 - 7s - loss: 1.5116 - accuracy: 0.5288
(pid=131343) Epoch 2/5
(pid=131342) 13/13 - 3s - loss: 1.4189 - accuracy: 0.5481
(pid=131342) Epoch 3/5
(pid=131343) 13/13 - 3s - loss: 1.5150 - accuracy: 0.5288
(pid=131343) Epoch 3/5
(pid=131342) 13/13 - 2s - loss: 1.3813 - accuracy: 0.4904
(pid=131342) Epoch 4/5
(pid=131343) 13/13 - 3s - loss: 1.5095 - accuracy: 0.5673
(pid=131343) Epoch 4/5
(pid=131342) 13/13 - 3s - loss: 1.3280 - accuracy: 0.5481
(pid=131342) Epoch 5/5
(pid=131343) 13/13 - 3s - loss: 1.4511 - accuracy: 0.4904
(pid=131343) Epoch 5/5
(pid=131342) 13/13 - 2s - loss: 1.2823 - accuracy: 0.6346
(pid=131343) 13/13 - 2s 

KeyboardInterrupt: ignored

In [23]:
from functools import reduce
#function and class to support implementation of FedCostWAvg
def aggregate_FedCost(results: List[Tuple[Weights, int]], alpha) -> Weights:
  """Compute weighted average."""
  # Calculate the total number of examples used during training
  num_examples_total = sum([num_examples for _, num_examples, _ in results])
  num_k_total = sum([k_val['metrics'] for _, _, k_val in results])


  # Create a list of weights, each multiplied by the related number of examples
  weighted_weights = []
  for weights, num_examples, k_val in results:

    weight_coef = ((alpha * (num_examples / num_examples_total)) + 
                   ((1-alpha) * k_val['metrics'] / num_k_total))
    

    weighted_weights.append([weight_coef*layer for layer in weights])

  weights_prime: Weights = [
      reduce(np.add, layer_updates)
      for layer_updates in zip(*weighted_weights)]

  return weights_prime


class FedCostWAvg(custom_FedAvg):
  def __init__(self, alpha_param: float = 0.5, 
               fraction_fit: float = 0.1,
               fraction_eval: float = 0.1,
               min_fit_clients: int = 1,
               min_eval_clients: int = 1,
               min_available_clients: int = 1,
               eval_fn: Optional[Callable[[Weights], Optional[Tuple[float, Dict[str, Scalar]]]]] = None,
               on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
               on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
               accept_failures: bool = True,
               initial_parameters: Optional[Parameters] = None,
               ) -> None:
    super().__init__(fraction_fit=fraction_fit,
            fraction_eval=fraction_eval,
            min_fit_clients=min_fit_clients,
            min_eval_clients=min_eval_clients,
            min_available_clients=min_available_clients,
            eval_fn=eval_fn,
            on_fit_config_fn=on_fit_config_fn,
            on_evaluate_config_fn=on_evaluate_config_fn,
            accept_failures=accept_failures,
            initial_parameters=initial_parameters
        )
    self.min_fit_clients = min_fit_clients
    self.min_eval_clients = min_eval_clients
    self.fraction_fit = fraction_fit
    self.fraction_eval = fraction_eval
    self.min_available_clients = min_available_clients
    self.eval_fn = eval_fn
    self.on_fit_config_fn = on_fit_config_fn
    self.on_evaluate_config_fn = on_evaluate_config_fn
    self.accept_failures = accept_failures
    self.pre_weights: Optional[Weights] = None
    self.alpha = alpha_param

  def aggregate_fit(self, rnd: int, results: List[Tuple[ClientProxy, FitRes]],failures: List[BaseException],) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
    """Aggregate fit results using weighted average."""
    if not results:
        return None, {}
    # Do not aggregate if there are failures and failures are not accepted
    if not self.accept_failures and failures:
        return None, {}
        
    # Convert results
    weights_results = [
        (parameters_to_weights(fit_res.parameters), fit_res.num_examples, fit_res.metrics)
        for client, fit_res in results
    ]
    return weights_to_parameters(aggregate_FedCost(weights_results, self.alpha)), {}

In [25]:
model = create_model()

model.compile(optimizer = 'sgd', loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Create FedCostWAvg strategy
strategy = FedCostWAvg(
        alpha_param = 0.5,
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_eval=0.1,  # Sample 10% of available clients for evaluation
        min_fit_clients=10,  # Never sample less than 10 clients for training
        min_eval_clients=10,  # Never sample less than 10 clients for evaluation
        min_available_clients=int(NUM_CLIENTS * 0.75),  # Wait until at least 75 clients are available
        eval_fn=get_eval_fn(model,NUM_CLIENTS))

# Start simulation
start = time()

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=ROUNDS,
    strategy=strategy)

end = time()

print("wall clock time was: " + str(end-start))

INFO flower 2021-12-08 22:16:02,272 | app.py:95 | Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3706479820.0, 'node:172.28.0.2': 1.0, 'memory': 7412959643.0}
INFO:flower:Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3706479820.0, 'node:172.28.0.2': 1.0, 'memory': 7412959643.0}
INFO flower 2021-12-08 22:16:02,284 | app.py:104 | Starting Flower simulation running: {'num_rounds': 50}
INFO:flower:Starting Flower simulation running: {'num_rounds': 50}
INFO flower 2021-12-08 22:16:02,290 | server.py:118 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2021-12-08 22:16:02,294 | server.py:304 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
INFO flower 2021-12-08 22:16:06,141 | server.py:307 | Received initial parameters from one random client
INFO:flower:Received initial parameters from one random client
INFO flower 2021-12-08 22:16:06,146 |

17/17 [==============================] - 1s 21ms/step - loss: 2.2968 - accuracy: 0.1121


INFO flower 2021-12-08 22:16:06,872 | server.py:127 | initial parameters (loss, other metrics): 2.296849489212036, {'accuracy': 0.11214952915906906}
INFO:flower:initial parameters (loss, other metrics): 2.296849489212036, {'accuracy': 0.11214952915906906}
INFO flower 2021-12-08 22:16:06,877 | server.py:133 | FL starting
INFO:flower:FL starting
DEBUG flower 2021-12-08 22:16:06,885 | server.py:255 | fit_round: strategy sampled 10 clients (out of 100)
DEBUG:flower:fit_round: strategy sampled 10 clients (out of 100)



server accuracy 0.11214952915906906
server loss 2.296849489212036

(pid=69814) 3/3 - 1s - loss: 2.2943 - accuracy: 0.1125
(pid=69814) Epoch 1/5
(pid=69814) 10/10 - 5s - loss: 2.3284 - accuracy: 0.1000
(pid=69814) Epoch 2/5
(pid=69815) 4/4 - 1s - loss: 2.3072 - accuracy: 0.0769
(pid=69815) Epoch 1/5
(pid=69814) 10/10 - 2s - loss: 2.3339 - accuracy: 0.1625
(pid=69814) Epoch 3/5
(pid=69814) 10/10 - 1s - loss: 2.3086 - accuracy: 0.1375
(pid=69814) Epoch 4/5
(pid=69814) 10/10 - 1s - loss: 2.3428 - accuracy: 0.1375
(pid=69814) Epoch 5/5
(pid=69814) 10/10 - 1s - loss: 2.3421 - accuracy: 0.1000
(pid=69814) 3/3 - 0s - loss: 2.3082 - accuracy: 0.0972
(pid=69814) Epoch 1/5
(pid=69815) 13/13 - 8s - loss: 2.3572 - accuracy: 0.0673
(pid=69815) Epoch 2/5
(pid=69815) 13/13 - 2s - loss: 2.3461 - accuracy: 0.0673
(pid=69815) Epoch 3/5
(pid=69815) 13/13 - 2s - loss: 2.3504 - accuracy: 0.0673
(pid=69815) Epoch 4/5
(pid=69814) 9/9 - 7s - loss: 2.3383 - accuracy: 0.0556
(pid=69814) Epoch 2/5
(pid=69814) 
(

DEBUG flower 2021-12-08 22:17:33,613 | server.py:264 | fit_round received 10 results and 0 failures
DEBUG:flower:fit_round received 10 results and 0 failures


...

(pid=69814) 13/13 - 1s - loss: 0.8786 - accuracy: 0.7692
17/17 [==============================] - 0s 26ms/step - loss: 0.4105 - accuracy: 0.8822


INFO flower 2021-12-08 23:30:55,325 | server.py:154 | fit progress: (50, 0.4105360209941864, {'accuracy': 0.8822429776191711}, 4488.439554010001)
INFO:flower:fit progress: (50, 0.4105360209941864, {'accuracy': 0.8822429776191711}, 4488.439554010001)
DEBUG flower 2021-12-08 23:30:55,328 | server.py:205 | evaluate_round: strategy sampled 10 clients (out of 100)
DEBUG:flower:evaluate_round: strategy sampled 10 clients (out of 100)



server accuracy 0.8822429776191711
server loss 0.4105360209941864

(pid=69814) 1/1 - 0s - loss: 0.0166 - accuracy: 1.0000
(pid=69815) 1/1 - 0s - loss: 3.7463 - accuracy: 0.8000
(pid=69814) 1/1 - 0s - loss: 0.0055 - accuracy: 1.0000
(pid=69815) 1/1 - 0s - loss: 0.2934 - accuracy: 0.8333
(pid=69814) 1/1 - 0s - loss: 0.0157 - accuracy: 1.0000
(pid=69815) 1/1 - 0s - loss: 2.6875e-04 - accuracy: 1.0000
(pid=69814) 1/1 - 0s - loss: 0.1495 - accuracy: 0.8000
(pid=69815) 1/1 - 0s - loss: 0.0315 - accuracy: 1.0000


DEBUG flower 2021-12-08 23:31:00,361 | server.py:214 | evaluate_round received 10 results and 0 failures
DEBUG:flower:evaluate_round received 10 results and 0 failures
INFO flower 2021-12-08 23:31:00,373 | server.py:172 | FL finished in 4493.488252039999
INFO:flower:FL finished in 4493.488252039999
INFO flower 2021-12-08 23:31:00,380 | app.py:119 | app_fit: losses_distributed [(1, 2.2976374846917613), (2, 2.2940577073530717), (3, 2.293454226580533), (4, 2.2807731920359084), (5, 2.2588179249029894), (6, 2.234762632621909), (7, 2.188936608809012), (8, 2.124007816498096), (9, 2.0064667567872165), (10, 1.8799115606382781), (11, 1.7500744547162737), (12, 1.5319084695407323), (13, 1.543942881481988), (14, 1.2975245186915765), (15, 1.0903758493336764), (16, 1.211433348832307), (17, 0.9448273171271596), (18, 1.0867190648008276), (19, 0.8685765372855323), (20, 0.8476941227912903), (21, 0.8043149503377768), (22, 0.7229548650128501), (23, 0.6787002938881255), (24, 0.77487668968164), (25, 0.660515

(pid=69814) 1/1 - 0s - loss: 0.0275 - accuracy: 1.0000
(pid=69815) 1/1 - 0s - loss: 0.0075 - accuracy: 1.0000
wall clock time was: 4502.444605588913
